In [ ]:
import requests
import json
import xml.etree.ElementTree as ElementTree
import xmltodict
import sqlite3
import mysql.connector
from kafka import KafkaProducer, KafkaConsumer
from kafka.admin import KafkaAdminClient, NewTopic
import json
import time

from config import CTA_API_KEY

## Arrivals API

In [ ]:
params = {'mapid': 40630,
         'stpid': 30122,
          'max': 3,
         'key': CTA_API_KEY,
         'outputType':'JSON'}

In [ ]:
response = requests.get("http://lapi.transitchicago.com/api/1.0/ttarrivals.aspx", params=params)

In [ ]:
response.json()

In [ ]:
root = ElementTree.fromstring(response.text)

In [ ]:
# child = 
root.getchildren()

In [ ]:
child.text

In [ ]:
def xml_to_tree(node):
    curr = {}
    eta_num = 1
    for child in node.getchildren():
        child_tag = child.tag
        if child.getchildren():
            if child_tag == 'eta':
                child_tag += f'_{eta_num}'
                eta_num += 1
            curr[child_tag] = xml_to_tree(child)
        else:
            curr[child_tag] = child.text
    
    return curr

In [ ]:
response.text

In [ ]:
print(xml_to_tree(root))

In [ ]:
data_dict = xmltodict.parse(response.content)

## Follow this train API

In [ ]:
params = {'runnumber':831,
         'key': CTA_API_KEY,
         'outputType':'JSON'}

In [ ]:
response = requests.get("https://lapi.transitchicago.com/api/1.0/ttfollow.aspx", params=params)

In [ ]:
response.json()

## Locations API

In [ ]:
params = {'rt':'red',
         'key': CTA_API_KEY,
         'outputType':'JSON'}

In [ ]:
response = requests.get("https://lapi.transitchicago.com/api/1.0/ttpositions.aspx", params=params)

In [ ]:
response.json()

## Database for storing train run delays per stop

### Sqlite

In [ ]:
conn = sqlite3.connect("cta_trains.db")
cur = conn.cursor()

In [ ]:
# Create train_delays table
cur.execute("""
create table train_delays
(train_line string, run_number int, day date,
previous_station string, next_station string, 
delay_in_minutes float)
""")

### MySQL

In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  port=3306,
  user="myuser",
  password="mypassword",
  database="mydatabase"
)

In [ ]:
mycursor = mydb.cursor()

mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

In [ ]:
mycursor.execute("""
create table train_delays
(train_line VARCHAR(10), run_number VARCHAR(10), day date,
previous_station VARCHAR(30), next_station VARCHAR(30), 
delay_in_minutes float, original_eta timestamp, latest_eta timestamp)
""")

In [ ]:
mycursor.execute("""
select *
from train_delays
""")

result = mycursor.fetchall()

for x in result:
    print(x)

In [ ]:
# mycursor.execute("""drop table train_delays""")

## Fetch latest train data

In [ ]:
def fetch_latest_etas(train_line: str):
    params = {'rt':train_line,
         'key': CTA_API_KEY,
         'outputType':'JSON'}
    
    response = requests.get("https://lapi.transitchicago.com/api/1.0/ttpositions.aspx", params=params)
    
    return response.json()

In [ ]:
fetch_latest_etas('red')

## Create Kafka topic

In [ ]:
admin_client = KafkaAdminClient(
    bootstrap_servers="localhost:9092",  # Replace with your Kafka broker address
#     client_id="my-admin-client"
)

topic_name = "train-etas"
num_partitions = 1
replication_factor = 1

new_topic = NewTopic(name=topic_name, num_partitions=num_partitions, replication_factor=replication_factor)

admin_client.create_topics([new_topic])

## Delete Kafka Topic

In [ ]:
admin_client = KafkaAdminClient(
    bootstrap_servers="localhost:9092",  # Replace with your Kafka broker address
#     client_id="my-admin-client"
)

topic_name = 'train-etas'

try:
    admin_client.delete_topics([topic_name])
    print(f"Topic '{topic_name}' deleted successfully.")
except Exception as e:
    print(f"Error deleting topic '{topic_name}': {e}")

## Set up Kafka producer

In [ ]:
producer = KafkaProducer(bootstrap_servers='localhost:9092',
                        value_serializer=lambda v: json.dumps(v).encode('utf-8'))

## Set up Kafka consumer to print messages in topic

In [ ]:
from kafka import KafkaConsumer

# To consume latest messages and auto-commit offsets
consumer = KafkaConsumer('train-etas',
                         bootstrap_servers=['localhost:9092'])

for message in consumer:
    print(message.value.decode('utf-8'))

## PyFlink 

In [ ]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors import FlinkKafkaConsumer
from pyflink.common.serialization import SimpleStringSchema
from pyflink.common.typeinfo import Types
import json

env = StreamExecutionEnvironment.get_execution_environment()

kafka_consumer = FlinkKafkaConsumer(
    topics=['train-etas'],
    deserialization_schema=SimpleStringSchema(),
    properties={'bootstrap.servers': 'localhost:9092'}
)

data_stream = env.add_source(kafka_consumer)

data_stream.print()

## Poll api and send

In [ ]:
def poll_api_and_send(train_line):
    while True:
        latest_data = fetch_latest_etas(train_line)
        producer.send('train-etas', latest_data)
        print('sent latest train eta data')
        
        time.sleep(30)

In [ ]:
poll_api_and_send('red')